# Optimizing

Running faster your code.

## Measuring times

In [12]:
import numpy as np
a = np.arange(1000)
%timeit a**2

The slowest run took 26881.47 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.64 µs per loop


## Profiling

In [19]:
'{:.100f}'.format(pi)

'3.1415926535897931159979634685441851615905761718750000000000000000000000000000000000000000000000000000'

In [2]:
%timeit map(lambda x: x^2, range(1000))

1000 loops, best of 3: 283 µs per loop


In [1]:
!python -m timeit -n 10000 'map(lambda x: x^2, range(1000))'

10000 loops, best of 3: 284 usec per loop


## Delegating in C
When you want to speed-up your code or simply when you need to reuse C code, it is possible to use from Python. There are several alternatives:

1. [Cython](http://cython.org/): A superset of Python to allow you call C functions and load Python variables with C ones. 
2. [SWIG (Simplified Wrapper Interface Generator)](http://www.swig.org/): A software development tool to connect C/C++ programs with other languages (included Python).
3. [Ctypes](http://python.net/crew/theller/ctypes/): A Python package that can be used to call shared libraries (`.ddl`/`.so`/`.dylib`) from Python.
4. [Python-C-API](https://docs.python.org/3.6/c-api/index.html): A low-level interface between (compiled) C code and Python.

We will show how to use Python-C-API because is the most flexible and efficient alternative. However, it is also the hardest to code.

### The C code to reuse in Python

In [2]:
!cat sum_array_lib.c

long int sum_array(double* a, int N) {
  int i;
  double sum = 0;
  for(i=0; i<N; i++) {
    sum += *a+i;
  }
  return sum;
}


In [4]:
!cat sum_array.c

#include <stdio.h>
#include <time.h>
#include "sum_array_lib.c"

#define N 100000

int main() {
  double a[N];
  int i;
  clock_t start, end;
  double cpu_time;
  for(i=0; i<N; i++) {
    a[i] = i;
  }
  start = clock();
  double sum = sum_array(a,N);
  end = clock();
  printf("%ld ", sum);
  cpu_time = ((double) (end - start)) / CLOCKS_PER_SEC;
  cpu_time *= 1000000;
  printf("%f usegs\n", cpu_time);
}


In [10]:
!gcc -O3 sum_array.c -o sum_array
!./sum_array

4999950000.000000 165.000000 usegs


### The module

In [11]:
!cat sum_array_module.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <Python.h>            /* Compulsory in every module */
#include <numpy/arrayobject.h> /* To interact with numpy arrays */
#include "sum_array_lib.c"

static PyObject* sumArray(PyObject* self, PyObject* args) {
  int N;
  long int sum;
  //int* a;
  PyArrayObject *in_array;
  
  clock_t start, end;
  double cpu_time;

  /*  parse the input */
  //if (!PyArg_ParseTuple(args, "i", &N))
  if (!PyArg_ParseTuple(args, "O!", &PyArray_Type, &in_array))
    return NULL;
  /* if the above function returns -1, an appropriate Python exception will
   * have been set, and the function simply returns NULL
   */

  N = PyArray_DIM(in_array, 0);
  printf("array size %d\n", N);

  npy_double* data  = (npy_double*)PyArray_DATA(in_array);
  //a = (int*)malloc(N*sizeof(int));
  //if (!a) return NULL;
  
  /*for(i=0; i<N; i++) {
    data[i] = i;
    }*/

  start = clock();
  sum = sum_array(data, N);
  end = clock();
  cpu_time = ((double) (

### Module compilation

In [12]:
!cat setup.py

from distutils.core import setup, Extension
import numpy.distutils.misc_util

# define the extension module
sum_array_module = Extension(
    'sum_array_module',
    sources=['sum_array_module.c'],
    include_dirs=numpy.distutils.misc_util.get_numpy_include_dirs()
)

# run the setup
setup(
    ext_modules=[sum_array_module],
)


In [16]:
!python setup.py build_ext --inplace

running build_ext
building 'sum_array_module' extension
C compiler: clang -fno-strict-aliasing -fno-common -dynamic -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes

compile options: '-I/usr/local/lib/python2.7/site-packages/numpy/core/include -I/usr/local/include -I/usr/local/opt/openssl/include -I/usr/local/opt/sqlite/include -I/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/include/python2.7 -c'
clang: sum_array_module.c
In file included from sum_array_module.c:5:
In file included from /usr/local/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /usr/local/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /usr/local/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1809:
/usr/local/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: "Using deprecated NumPy API, disable it by "          "#defining NPY_NO_DEP

In [3]:
import sum_array_module
import numpy as np
a = np.arange(100000)
%timeit sum_array_module.sumArray(a)

204 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
